In [1]:
import pandas as pd
import spacy
from sklearn import model_selection
import numpy as np


from tensorflow import keras
from tensorflow.keras import layers, models

In [2]:
df = pd.read_json(open("../../data/data.json", "r", encoding="utf8"))

In [3]:
df

,title,author,year,text,themes
0,"[45, 4, 141, 903]",0,2002,"[2, 2334, 1097, 5, 2334, 298, 9780, 1002, 412,...","[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,[2828],1,2002,"[1, 95, 106, 4385, 3, 144, 291, 33, 1426, 8803...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,[209],2,2002,"[106, 20, 719, 21, 1474, 329, 240, 159, 421, 2...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,"[1210, 40, 114, 1989, 18, 2321]",3,2002,"[329, 303, 17, 70, 8, 1, 8, 1, 43, 1345, 1, 20...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,"[259, 1210]",4,2002,"[9, 649, 76, 140, 544, 206, 679, 179, 107, 44,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...,...
17067,"[151, 18, 2, 5178]",168,1990,"[1, 395, 2499, 331, 545, 8944, 28, 2573, 93, 8...","[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, ..."
17068,[],1172,1990,"[2, 597, 152, 153, 200, 277, 167, 4114, 3080, ...","[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, ..."
17070,"[15, 114]",3191,1990,"[114, 17, 70, 125, 628, 2, 797, 1, 99, 1133, 1...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
17071,"[15, 25, 1, 13]",3434,2005,"[15, 1, 13, 37, 36, 15, 1, 311, 174, 544, 38, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [4]:
# one hot encoding for authors
def encode_authors(author_code):
    qty = df.author.max()
    result = [0] * (qty + 1)
    result[author_code] = 1
    return result

df.author = df.author.apply(encode_authors)

In [5]:
max_features = 10000  # maximum number of words in vocabulari 5000
max_len = 150  # max length of string

In [6]:
df['joined_text'] = df['text'] + df['title']
df['X2'] = df['themes'] + df['author'] 
train_df, test_df = model_selection.train_test_split(df, test_size=0.1, random_state=42)
X1_train = keras.preprocessing.sequence.pad_sequences(train_df['joined_text'].to_list(), maxlen=max_len, padding='post')
X2_train = np.stack(train_df['X2'])
Y_train = np.stack(train_df['year'])
X1_test = keras.preprocessing.sequence.pad_sequences(list(test_df['joined_text']), maxlen=max_len, padding='post')
X2_test = np.stack(test_df['X2'])
Y_test = np.stack(test_df['year'])
#X = keras.preprocessing.sequence.pad_sequences(list(joined_text), maxlen=max_len, padding='post')
#Y = np.array(df['year'].to_list())

In [7]:
#X_train, X_test, Y_train, Y_test = model_selection.train_test_split(X, Y, test_size=0.1, random_state=42)

### Model 

In [8]:
embedding_dim =64
model1 = keras.models.Sequential([
  keras.layers.Embedding(input_dim=max_features,
                           output_dim=embedding_dim,
                           input_length=max_len),
  keras.layers.Flatten(),
  keras.layers.Dense(2000,activation='relu'),
  keras.layers.Dense(500,activation='relu'),
  keras.layers.Dense(100,activation='relu'),
  keras.layers.Dense(1, activation='relu')
])

model1.compile(optimizer='nadam',
              loss='mean_squared_error',
              metrics=['MAE'])

model1.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 150, 64)           640000    
_________________________________________________________________
flatten (Flatten)            (None, 9600)              0         
_________________________________________________________________
dense (Dense)                (None, 2000)              19202000  
_________________________________________________________________
dense_1 (Dense)              (None, 500)               1000500   
_________________________________________________________________
dense_2 (Dense)              (None, 100)               50100     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 101       
Total params: 20,892,701
Trainable params: 20,892,701
Non-trainable params: 0
____________________________________________

In [10]:
epochs = 5
model1.fit(X1_train, Y_train,
          #batch_size=128,
          validation_data=(X1_test,Y_test),
          epochs=epochs)

Epoch 1/5
385/385 [==============================] - 85s 221ms/step - loss: 14420.6348 - MAE: 94.6558 - val_loss: 39660.5938 - val_MAE: 184.6050
Epoch 2/5
385/385 [==============================] - 84s 219ms/step - loss: 11623.7041 - MAE: 85.1576 - val_loss: 31679.7227 - val_MAE: 167.6673
Epoch 3/5
385/385 [==============================] - 83s 216ms/step - loss: 9095.6572 - MAE: 75.5035 - val_loss: 4228.8892 - val_MAE: 45.4523
Epoch 4/5
385/385 [==============================] - 85s 220ms/step - loss: 9051.0430 - MAE: 77.4146 - val_loss: 8560.8301 - val_MAE: 76.9370
Epoch 5/5
385/385 [==============================] - 84s 218ms/step - loss: 7051.2827 - MAE: 66.5734 - val_loss: 4184.0913 - val_MAE: 44.9800


In [11]:
score1 = model1.evaluate(np.array(X1_test), np.array(Y_test)) 

print("Test Score:", score1[0])
print("Test Accuracy:", score1[1])

43/43 [==============================] - 1s 15ms/step - loss: 4184.0913 - MAE: 44.9800
Test Score: 4184.09130859375
Test Accuracy: 44.97998809814453


In [ ]:
#X2 = np.array([row.author + row.themes for index, row in df.iterrows()])
#Y2 = np.array(df.year)

In [12]:

model2 = keras.models.Sequential([
  keras.layers.Dense(units=2000, activation='relu', input_shape=[len(X2_train[0])]),
  
  keras.layers.Dense(2000,activation='relu'),
  keras.layers.Dense(500,activation='relu'),
  keras.layers.Dense(100,activation='relu'),
  keras.layers.Dense(1, activation='relu')
])

model2.build()

In [13]:
model2.compile(optimizer='nadam',
              loss='mean_squared_error',
              metrics=['MAE'])

In [14]:
model2.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 2000)              8234000   
_________________________________________________________________
dense_5 (Dense)              (None, 2000)              4002000   
_________________________________________________________________
dense_6 (Dense)              (None, 500)               1000500   
_________________________________________________________________
dense_7 (Dense)              (None, 100)               50100     
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 101       
Total params: 13,286,701
Trainable params: 13,286,701
Non-trainable params: 0
_________________________________________________________________


In [15]:
#X2_train, X2_test, Y2_train, Y2_test = model_selection.train_test_split(X2, Y2, test_size=0.1, random_state=42)


In [16]:
model2.fit(X2_train, Y_train, epochs=5, validation_split=0.1)

Epoch 1/5
347/347 [==============================] - 41s 119ms/step - loss: 188536.6250 - MAE: 235.9512 - val_loss: 29597.2949 - val_MAE: 130.8318
Epoch 2/5
347/347 [==============================] - 41s 119ms/step - loss: 31507.2129 - MAE: 138.4695 - val_loss: 16479.7461 - val_MAE: 102.7494
Epoch 3/5
347/347 [==============================] - 41s 119ms/step - loss: 20937.9746 - MAE: 112.0772 - val_loss: 14742.3340 - val_MAE: 99.0083
Epoch 4/5
347/347 [==============================] - 42s 120ms/step - loss: 15904.2559 - MAE: 98.6465 - val_loss: 8413.0625 - val_MAE: 72.2343
Epoch 5/5
347/347 [==============================] - 41s 118ms/step - loss: 12306.5586 - MAE: 84.9272 - val_loss: 6978.4385 - val_MAE: 64.3089


In [17]:
score2 = model2.evaluate(np.array(X2_test), np.array(Y_test)) 

print("Test Score:", score2[0])
print("Test Accuracy:", score2[1])

43/43 [==============================] - 0s 11ms/step - loss: 7557.6396 - MAE: 67.3377
Test Score: 7557.6396484375
Test Accuracy: 67.3377456665039


In [ ]:
y_pred1 = model1.predict(X1_test)
y_pred2 = model2.predict(X2_test)
result = (y_pred1 + y_pred2)/2

In [ ]:

keras.metrics.mean_absolute_error(
    y_pred1.flatten(), y_pred2.flatten()
)



In [ ]:
output_dim = 100

inputs = keras.Input(shape=(None,), dtype="int64")

# Next, we add a layer to map those vocab indices into a space of dimensionality
# 'embedding_dim'.
x = layers.Embedding(max_features, output_dim)(inputs)
x = layers.Dropout(0.5)(x)

# Conv1D + global max pooling
x = layers.Conv1D(128, 7, padding="valid", activation="relu", strides=3)(x)
x = layers.Conv1D(128, 7, padding="valid", activation="relu", strides=3)(x)
x = layers.GlobalMaxPooling1D()(x)

# We add a vanilla hidden layer:
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.5)(x)

predictions = layers.Dense(1, activation='relu', name="predictions")(x)

model3 = keras.Model(inputs, predictions)

# Compile the model with binary crossentropy loss and an adam optimizer.
model3.compile(loss="mean_squared_error", optimizer="adam", metrics=["MAE"])


In [ ]:
model3.summary()

In [ ]:
batch_size = 64
epochs = 5
model3.fit(X1_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1)

In [ ]:
accr = model3.evaluate(X1_test,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))